Create new database, new table

In [1]:
# region import
import pyodbc
import pandas as pd
import MetaTrader5 as mt5
from sqlalchemy import create_engine

from helpers_database import *
# endregion

# region class Account, Pair, Server
class Account:
    def __init__(self, login, password, server):
        self.login = login
        self.password = password
        self.server = server

class Pair:
    def __init__(self, symbol, timeframe):
        self.symbol = symbol
        self.timeframe = timeframe

class Create_Server:
    def __init__(self, driver, server, username, password, db_name):
        self.driver = driver
        self.server = server
        self.username = username
        self.password = password
        self.db_name =db_name
# endregion

# region variables
# Define the connection parameters for SQL Server
server = 'HUNG-LAPTOP' 
username = 'trading'  
password = 'Dulieulon123'
driver = '{ODBC Driver 17 for SQL Server}'
driver_alchemy = 'ODBC+Driver+17+for+SQL+Server'
db_name = "audusd"
table_name = "audusd_m15"
pair_name = "AUDUSD.sml"
timeframe = mt5.TIMEFRAME_M15
unique_text = 'text_id'
from_day = pd.to_datetime("2000-01-01 00:00:00")
to_day = pd.to_datetime("2025-01-01 00:00:00")
chunk_capacity = 10000
# endregion

Server = Create_Server(driver, server, username, password, db_name)
create_table(Server, table_name)

# take data from OANDA MT5, split into smaller chunks for processing
account = Account(6269482, 'jc6qhduy', 'OANDA-Demo-1')
pair = Pair(pair_name, timeframe)
date_ranges = generate_date_ranges(2000, 2025)
ohlc_df = get_data_MT5(link, account, pair, from_day, to_day)
print("number of rows from mt5: ", len(ohlc_df))
df_chunk = divide_dataframe(ohlc_df, chunk_capacity)

# add data to the table
engine = create_engine(f"mssql+pyodbc://{Server.username}:{Server.password}@{Server.server}/{Server.db_name}?driver={driver_alchemy}")
add_data(table_name, engine, df_chunk)


Connection successful!
Database mt5_trade already exists.
Table audusd_m15 checked and created if it did not exist.
Database 'mt5_trade' and table 'audusd_m15' are connected.


d:\trading-with-python\Sen_trading\Database\helpers_database.py:126: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])


number of rows from mt5:  6495
Chunk 1 data imported successfully into audusd_m15!
Chunk 2 data imported successfully into audusd_m15!
Chunk 3 data imported successfully into audusd_m15!
Chunk 4 data imported successfully into audusd_m15!
Chunk 5 data imported successfully into audusd_m15!
Chunk 6 data imported successfully into audusd_m15!
Chunk 7 data imported successfully into audusd_m15!
Data import completed successfully.


In [7]:
from tvDatafeed import TvDatafeed, Interval

# Initialize TvDatafeed with your TradingView username and password
username = 'ngdanghung2000@gmail.com'
password = 'Matkhaumoi123!'

for i in (1,30):
    tv = TvDatafeed(username, password)

    # Download historical data for a specific symbol
    symbol = 'SPX500USD'
    exchange = 'OANDA'
    interval = Interval.in_1_minute
    n_bars = 10000
    data = tv.get_hist(symbol, exchange, interval, n_bars)

print(data)


ERROR:tvDatafeed.main:error while signin


                              symbol    open    high     low   close  volume
datetime                                                                    
2024-12-23 06:00:00  OANDA:SPX500USD  5974.8  5975.0  5973.6  5973.6     9.0
2024-12-23 06:01:00  OANDA:SPX500USD  5973.6  5976.6  5973.0  5976.6   157.0
2024-12-23 06:02:00  OANDA:SPX500USD  5977.2  5979.2  5975.6  5977.6   167.0
2024-12-23 06:03:00  OANDA:SPX500USD  5978.0  5979.4  5977.2  5978.0    75.0
2024-12-23 06:04:00  OANDA:SPX500USD  5977.8  5977.8  5974.0  5976.4    84.0
...                              ...     ...     ...     ...     ...     ...
2024-12-31 17:19:00  OANDA:SPX500USD  5933.8  5934.4  5933.8  5934.2     9.0
2024-12-31 17:20:00  OANDA:SPX500USD  5934.8  5935.2  5934.8  5935.0     6.0
2024-12-31 17:21:00  OANDA:SPX500USD  5934.4  5935.2  5934.4  5935.2    15.0
2024-12-31 17:22:00  OANDA:SPX500USD  5935.2  5935.2  5935.0  5935.2    10.0
2024-12-31 17:23:00  OANDA:SPX500USD  5935.0  5935.0  5935.0  5935.0     1.0

### GET DATA FROM SQL, we have to split the data into smaller chunks to prevent memory allocate error.

In [1]:
# GET DATA FROM SQL
# region import
import pyodbc
import pandas as pd
import MetaTrader5 as mt5
from sqlalchemy import create_engine

# Define the connection parameters for SQL Server
server = 'HUNG-LAPTOP' 
username = 'trading'  
password = 'Dulieulon123'
driver = '{ODBC Driver 17 for SQL Server}'
driver_alchemy = 'ODBC+Driver+17+for+SQL+Server'
db_name = "audusd"
table_name = "audusd_m15"
pair_name = "AUDUSD.sml"
timeframe = mt5.TIMEFRAME_M15
unique_text = 'text_id'
from_day = pd.to_datetime("2000-01-01 00:00:00")
to_day = pd.to_datetime("2025-01-01 00:00:00")
chunk_capacity = 50000
# endregion

class Create_Server:
    def __init__(self, driver, server, username, password, db_name):
        self.driver = driver
        self.server = server
        self.username = username
        self.password = password
        self.db_name =db_name
Server = Create_Server(driver, server, username, password, db_name)

# Create the SQLAlchemy engine
engine = create_engine(f"mssql+pyodbc://{Server.username}:{Server.password}@{Server.server}/{Server.db_name}?driver={driver_alchemy}")

# Define the query to fetch data from the table `audusd_m15`
query = f"SELECT * FROM {table_name}"

# Specify the chunk size
chunk_size = 100000  # Adjust this based on your memory limits

# Read the data in chunks
chunk_iter = pd.read_sql_query(query, engine, chunksize=chunk_size)

# Initialize an empty list to store each chunk
chunks = []

# Process each chunk
for chunk in chunk_iter:
    # Perform any necessary data processing on the chunk here
    chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunks, ignore_index=True)

# Now you can work with your DataFrame `df`
print(df.head(1))


   id       time    open    high     low  close  tick_volume  spread  \
0   1 2000-01-03  0.6563  0.6574  0.6563  0.657           11      50   

            text_id  
0  2000-01-03-00-00  


In [17]:
import MetaTrader5 as mt5
import pandas as pd

timeframe_list = [mt5.TIMEFRAME_M1, mt5.TIMEFRAME_M2, mt5.TIMEFRAME_M3, mt5.TIMEFRAME_M4,
                  mt5.TIMEFRAME_M5, mt5.TIMEFRAME_M10, mt5.TIMEFRAME_M15,
                  mt5.TIMEFRAME_M30, mt5.TIMEFRAME_H1, mt5.TIMEFRAME_H2,
                  mt5.TIMEFRAME_H3, mt5.TIMEFRAME_H4, mt5.TIMEFRAME_H6, mt5.TIMEFRAME_D1, 
                  mt5.TIMEFRAME_W1, mt5.TIMEFRAME_MN1]

prefix = "audusd"
table_name_list = [f"{prefix}_m1", f"{prefix}_m2", f"{prefix}_m3", f"{prefix}_m4",
                   f"{prefix}_m5", f"{prefix}_m10", f"{prefix}_m15",
                   f"{prefix}_m30", f"{prefix}_h1", f"{prefix}_h2",
                   f"{prefix}_h3", f"{prefix}_h4", f"{prefix}_h6", f"{prefix}_d1", 
                   f"{prefix}_w1", f"{prefix}_mn1"]


# Create a dictionary with the two lists
data = {
    "Timeframe": timeframe_list,
    "Table Name": table_name_list
}

# Create DataFrame
df_id = pd.DataFrame(data)

for index, row in df_id.iterrows():
    print(row['Timeframe'], ", ", row['Table Name'])


1 ,  audusd_m1
2 ,  audusd_m2
3 ,  audusd_m3
4 ,  audusd_m4
5 ,  audusd_m5
10 ,  audusd_m10
15 ,  audusd_m15
30 ,  audusd_m30
16385 ,  audusd_h1
16386 ,  audusd_h2
16387 ,  audusd_h3
16388 ,  audusd_h4
16390 ,  audusd_h6
16408 ,  audusd_d1
32769 ,  audusd_w1
49153 ,  audusd_mn1
